In [1]:
import numpy as np
import helpers as hp

# Load Data

In [2]:
embed_word_metrix = np.load('./embeddings.npy')

In [3]:
metrix = np.load('./cooc.pkl')

In [4]:
metrix.shape

(21161, 21161)

In [5]:
vocab = np.load('./vocab.pkl')

In [14]:
train_all = hp.build_train(vocab,embed_word_metrix)

In [15]:
print(train_all.shape)

(200000, 21)


In [16]:
s_me_test = hp.sentence_metrix("./twitter-datasets/test_data.txt",vocab,embed_word_metrix)

In [17]:
s_me_test = np.asarray(s_me_test)

# LeastSquare

In [31]:
from sklearn import linear_model

In [13]:
reg = linear_model.LinearRegression()
reg.fit(s_me,y_train)

/usr/local/lib/python3.5/site-packages/scipy/linalg/basic.py:884: RuntimeWarning: internal gelsd driver lwork query error, required iwork dimension not returned. This is likely the result of LAPACK bug 0038, fixed in LAPACK 3.2.2 (released July 21, 2010). Falling back to 'gelss' driver.
  warnings.warn(mesg, RuntimeWarning)


LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [19]:
w_ls = np.insert(reg.coef_,0,reg.intercept_)

In [23]:
sme_ls_test = np.insert(s_me_test,0,1,axis=1)

In [25]:
test_y = sme_ls_test.dot(w_ls)

array([-0.12331972, -0.16652336, -0.36356422, ..., -0.15282616,
       -0.09943389, -0.25174843])

In [27]:
y_label = np.zeros(test_y.shape)

In [28]:
for i in range(0,10000,1):
    if (test_y[i]>0):
        y_label[i]  = 1
    else:
        y_label[i] = -1

In [29]:
y_label

array([-1., -1., -1., ..., -1., -1., -1.])

In [31]:
OUTPUT_PATH = './data/first.csv' 
hp.create_csv_submission(range(1,10001,1), y_label, OUTPUT_PATH)
#first try using least square (54.660%)

# SVM

In [30]:
from sklearn.svm import SVC

In [31]:
clf = SVC()

In [ ]:
clf.fit(s_me,y_train)

In [205]:
svm_test_y = clf.predict(s_me_test)

In [207]:
svm_test_y.shape

(10000,)

In [210]:
OUTPUT_PATH = './data/5.csv' 
hp.create_csv_submission(range(1,10001,1), svm_test_y, OUTPUT_PATH)
#first try using least square (54.440%)

## k-fold cross validation

In [27]:
from sklearn.model_selection import KFold

In [53]:
x_sample,y_sample = hp.build_sample(train_all,25000)

In [45]:
clf = SVC()

In [46]:
clf.fit(x_sample,y_sample)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [47]:
svm_test_y = clf.predict(s_me_test)

In [52]:
OUTPUT_PATH = './data/6.csv' 
hp.create_csv_submission(range(1,10001,1), svm_test_y, OUTPUT_PATH)
#first try using least square (54.220%)

In [43]:
kf_SVC = KFold(n_splits=5)

In [54]:
for train,test in kf_SVC.split(x_sample,y_sample):
    X_train, X_test, y_train, y_test = x_sample[train], x_sample[test], y_sample[train], y_sample[test]
    clff = SVC().fit(X_train,y_train)
    x = clff.score(X_test,y_test)
    print(x)

0.6068
0.6124
0.6152
0.6104
0.6018


# Matrix Factorization

In [13]:
from sklearn.decomposition import NMF

In [14]:
metrix.shape

(21161, 21161)

In [15]:
mf = NMF(n_components=20,init='random',random_state=0,max_iter=20000,alpha=0.25,eta=0.001)

In [16]:
mf.fit(metrix)

NMF(alpha=0.25, beta=1, eta=0.001, init='random', l1_ratio=0.0,
  max_iter=20000, n_components=20, nls_max_iter=2000, random_state=0,
  shuffle=False, solver='cd', sparseness=None, tol=0.0001, verbose=0)

In [17]:
H = mf.components_

In [18]:
H.shape

(20, 21161)

In [20]:
W = mf.transform(metrix)

In [21]:
W.shape

(21161, 20)

In [22]:
s_metrix_pos_d = hp.sentence_metrix("./twitter-datasets/pos_train.txt", vocab,W)
s_metrix_neg_d= hp.sentence_metrix("./twitter-datasets/neg_train.txt",vocab,W)

In [23]:
s_me_d = np.asarray(s_metrix_pos_d+s_metrix_neg_d)

In [24]:
train_all = np.insert(s_me_d,0,Y_train,axis=1)

In [25]:
x_sample_d,y_sample_d = hp.build_sample(train_all,25000)

In [32]:
clf = SVC()
clf.fit(x_sample_d,y_sample_d)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [33]:
kf_SVC = KFold(n_splits=5)

In [35]:
for train,test in kf_SVC.split(x_sample_d,y_sample_d):
    X_train, X_test, y_train, y_test = x_sample_d[train], x_sample_d[test], y_sample_d[train], y_sample_d[test]
    clff = SVC().fit(X_train,y_train)
    x = clff.score(X_test,y_test)
    print(x)

0.659
0.6596
0.6698
0.6648
0.6684


In [38]:
s_me_test_d = hp.sentence_metrix("./twitter-datasets/test_data.txt",vocab,W)

In [39]:
s_me_test_d = np.asarray(s_me_test_d)

In [40]:
svm_test_y_d = clf.predict(s_me_test_d)

In [41]:
OUTPUT_PATH = './data/7.csv' 
hp.create_csv_submission(range(1,10001,1), svm_test_y_d, OUTPUT_PATH)
#first try using least square (62.080%)